In [25]:
import os
import openai
import pandas as pd
import tiktoken
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone

In [26]:
load_dotenv()
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv('FINE_TUNED_TOXIC_DETECTION_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_index = os.getenv('PINECONE_GPT')

In [27]:
# initializing Pinecone vector database instance
docSearch = Pinecone(
    index_name=pc_index,
    embedding=OpenAIEmbeddings()
)

In [28]:
# query_result = docSearch.max_marginal_relevance_search("Dude need chill reverting Thats point annoying", k=3, fetch_k=50)
# query_result[0].page_content

In [29]:
def shot_additions(examples):
   evaluation_prompt = ''
   # Read the evaluation prompt from the text file with utf-8 encoding
   with open("../data/text/gpt-3-5-0125-two-shot-prompt.txt", "r", encoding="utf-8") as file:
        evaluation_prompt += file.read() 
   evaluation_prompt += '\n\n'
   for example in examples:
       evaluation_prompt += example

   return evaluation_prompt

In [30]:
def rag_prompt(text):
    two_shots = []
    search_results = docSearch.max_marginal_relevance_search(text, k=25, fetch_k=50)
    for i in search_results:
        content = i.page_content
        if i != 0 and len(content) < 50:
            two_shots.append(content)
    return shot_additions(two_shots)


In [31]:
rag_prompt("Dude need chill reverting Thats point annoying")

'You are tasked with classifying comments as either toxic or non-toxic from a non-biased viewpoint. \nA comment is toxic if it contains harmful, abusive, or offensive language, such as threats, hate speech, \npersonal attacks, obscene language, or discrimination. Label toxic comments with a 1 and non-toxic comments with a 0. \n\nMild criticism or disagreement without harmful language should be labeled as non-toxic (0). \nUse these guidelines:\n- Label toxic (1) if: profanity, slurs, threats, or discrimination is present.\n- Label non-toxic (0) if: the comment is neutral, polite, or constructive, even if it expresses disagreement.\n\nExamples for Reference:\n\n'

In [32]:
df = pd.read_csv('../data/csv/unprocessed_toxic_classification_dataset_short.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(int(x)))
df

,Text,Toxic
0,"Geez, are you forgetful! We've already discus...",0
1,Carioca RFA \r\n\r\nThanks for your support on...,0
2,"""\r\n\r\n Birthday \r\n\r\nNo worries, It's wh...",0
3,Pseudoscience category? \r\n\r\nI'm assuming t...,0
4,"(and if such phrase exists, it would be provid...",0
...,...,...
995,2006 (UTC)\r\n\r\n Not that any of the rest of...,0
996,"""\r\n\r\n Yay, let's all do the Pedantic Seman...",0
997,no am not how am I supposed to Know?,0
998,If you guys really discuss napoleon you need t...,1


In [33]:
fine_tuned = OpenAI(
    api_key=FINE_TUNED_TOXIC_DETECTION_API_KEY
)

In [34]:
tuned = pd.DataFrame(columns=['Text', 'Toxic'])

In [35]:
evaluation_prompt = ''
# Read the evaluation prompt from the text file with utf-8 encoding
with open("../data/text/gpt-3-5-0125-two-shot-prompt.txt", "r", encoding="utf-8") as file:
    evaluation_prompt += file.read()

In [36]:
evaluation_prompt

'You are tasked with classifying comments as either toxic or non-toxic from a non-biased viewpoint. \nA comment is toxic if it contains harmful, abusive, or offensive language, such as threats, hate speech, \npersonal attacks, obscene language, or discrimination. Label toxic comments with a 1 and non-toxic comments with a 0. \n\nMild criticism or disagreement without harmful language should be labeled as non-toxic (0). \nUse these guidelines:\n- Label toxic (1) if: profanity, slurs, threats, or discrimination is present.\n- Label non-toxic (0) if: the comment is neutral, polite, or constructive, even if it expresses disagreement.\n\nExamples for Reference:'

In [37]:
for index, row in df.iterrows():
    text = row['Text']
    completion = fine_tuned.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::ANEFVZOZ",
        messages=[
            {
                "role": "system", 
                "content": rag_prompt(text)
            },
            {
                "role": "user",
                "content": text
            }
        ]
    )
    tuned.loc[index] = [text, completion.choices[0].message.content]


In [38]:
tuned

,Text,Toxic
0,"Geez, are you forgetful! We've already discus...",0
1,Carioca RFA \r\n\r\nThanks for your support on...,0
2,"""\r\n\r\n Birthday \r\n\r\nNo worries, It's wh...",0
3,Pseudoscience category? \r\n\r\nI'm assuming t...,0
4,"(and if such phrase exists, it would be provid...",0
...,...,...
995,2006 (UTC)\r\n\r\n Not that any of the rest of...,0
996,"""\r\n\r\n Yay, let's all do the Pedantic Seman...",0
997,no am not how am I supposed to Know?,0
998,If you guys really discuss napoleon you need t...,1


In [39]:
compare = tuned['Toxic'] == df['Toxic']
gptthreefive_original_accuracy = compare.values.sum() / compare.size

In [40]:
compare

0       True
1       True
2       True
3       True
4       True
       ...  
995     True
996     True
997     True
998     True
999    False
Name: Toxic, Length: 1000, dtype: bool

In [41]:
# GPT-4o: 97.30% with too mid-detailed evaluation prompt
# GPT-3.5 turbo 50 set: 91.80% with summarized two-shot prompt costing 73 cents
# GPT-3.5 turbo 200 set: 92.00% with summarized two-shot prompt costing 72 cents
# GPT 3.5 turbo 2000 set: 97:40% with summarized two-shot prompt costing 72 cents
print(f"Accuracy: {gptthreefive_original_accuracy * 100:.2f}%")

Accuracy: 97.60%


In [42]:

# completion = fine_tuned.chat.completions.create(
#     model="ft:gpt-4o-2024-08-06:personal::AENT6Q3Y",
#     messages=[
#         {
#             "role": "system", 
#             "content": "The user has a toxicity threshold of 1, meaning that they find 90 to 100 percent of comments non-toxic and have extremely high tolerance. Label this text as either 0 for non-toxic or 1 for toxic."
#         },
#         {
#             "role": "user",
#             "content": df['Text'][998]
#         }
#     ]
# )

In [43]:
completion.choices[0].message.content

'1'

In [44]:
def count_tokens(prompt, model="gpt-3.5-turbo"):
    # Initialize the tiktoken encoder for your model
    encoding = tiktoken.encoding_for_model(model)
    # Encode the prompt to get the number of tokens
    tokens = encoding.encode(prompt)
    return len(tokens)